# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import datetime
from sql_queries import *

In [2]:
# run the "create_tables.py" script in order to create the sparkifydb database an all the necessary tables
! python3 create_tables.py

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('./data/song_data/')

In [6]:
song_files

['/home/workspace/data/song_data/A/B/C/TRABCRU128F423F449.json',
 '/home/workspace/data/song_data/A/B/C/TRABCTK128F934B224.json',
 '/home/workspace/data/song_data/A/B/C/TRABCUQ128E0783E2B.json',
 '/home/workspace/data/song_data/A/B/C/TRABCIX128F4265903.json',
 '/home/workspace/data/song_data/A/B/C/TRABCYE128F934CE1D.json',
 '/home/workspace/data/song_data/A/B/C/TRABCEC128F426456E.json',
 '/home/workspace/data/song_data/A/B/C/TRABCXB128F4286BD3.json',
 '/home/workspace/data/song_data/A/B/C/TRABCAJ12903CDFCC2.json',
 '/home/workspace/data/song_data/A/B/C/TRABCPZ128F4275C32.json',
 '/home/workspace/data/song_data/A/B/C/TRABCKL128F423A778.json',
 '/home/workspace/data/song_data/A/B/C/TRABCEI128F424C983.json',
 '/home/workspace/data/song_data/A/B/C/TRABCFL128F149BB0D.json',
 '/home/workspace/data/song_data/A/B/B/TRABBBV128F42967D7.json',
 '/home/workspace/data/song_data/A/B/B/TRABBLU128F93349CF.json',
 '/home/workspace/data/song_data/A/B/B/TRABBXU128F92FEF48.json',
 '/home/workspace/data/so

In [7]:
filepath = song_files[0]

In [8]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8IEZO1187B99055E,NaN,,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
# Test if songs table exists and if it is empty as it should
cur.execute("select count(*) from songs;")
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(0,)


In [11]:
cur.execute(song_table_insert, song_data)
conn.commit()

In [12]:
# Check if the insert statement worked as expected
cur.execute("select * from songs;")
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404)


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [13]:
df[['artist_id', 'artist_name',  'artist_location',  'artist_latitude',  'artist_longitude']].values[0].tolist()

['AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan]

In [14]:
artist_data = df[['artist_id', 'artist_name',  'artist_location',  'artist_latitude',  'artist_longitude']].values[0].tolist()
artist_data

['AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [15]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

In [16]:
cur.execute("select * from artists;")
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan)


# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [17]:
log_files = get_files('./data/log_data/')

In [18]:
log_files

['/home/workspace/data/log_data/2018/11/2018-11-29-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-27-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-11-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-18-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-08-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-30-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-17-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-20-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-28-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-06-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-23-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-13-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-21-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-16-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-04-events.json',
 '/home/workspace/data/lo

In [19]:
filepath = log_files[0]

In [20]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [21]:
df.shape

(366, 18)

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [22]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [23]:
df = df[df['page']=='NextSong'].reset_index()
df.head()

,index,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [24]:
df.shape

(319, 19)

In [25]:
#pd.to_datetime(df.ts, unit='ms')

In [26]:
t = pd.to_datetime(df.ts, unit='ms')
t.head()

0   2018-11-29 00:00:57.796
1   2018-11-29 00:01:30.796
2   2018-11-29 00:04:01.796
3   2018-11-29 00:04:55.796
4   2018-11-29 00:07:13.796
Name: ts, dtype: datetime64[ns]

In [27]:
#t.dt.hour
#t.dt.day
#t.dt.week
#t.dt.month
#t.dt.year
#t.dt.day_name()

In [28]:
time_data = (t, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.day_name())
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [29]:
column_labels

('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

In [30]:
time_df = pd.DataFrame(dict(zip(column_labels, time_data)))
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-29 00:00:57.796,0,29,48,11,2018,Thursday
1,2018-11-29 00:01:30.796,0,29,48,11,2018,Thursday
2,2018-11-29 00:04:01.796,0,29,48,11,2018,Thursday
3,2018-11-29 00:04:55.796,0,29,48,11,2018,Thursday
4,2018-11-29 00:07:13.796,0,29,48,11,2018,Thursday


In [31]:
time_df.dtypes

start_time    datetime64[ns]
hour                   int64
day                    int64
week                   int64
month                  int64
year                   int64
weekday               object
dtype: object

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [32]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

In [33]:
# Check if all rows inserted in the relevant table as expected
cur.execute("select * from time;")
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(datetime.datetime(2018, 11, 29, 0, 0, 57, 796000), 0, 29, 48, 11, 2018, 'Thursday')
(datetime.datetime(2018, 11, 29, 0, 1, 30, 796000), 0, 29, 48, 11, 2018, 'Thursday')
(datetime.datetime(2018, 11, 29, 0, 4, 1, 796000), 0, 29, 48, 11, 2018, 'Thursday')
(datetime.datetime(2018, 11, 29, 0, 4, 55, 796000), 0, 29, 48, 11, 2018, 'Thursday')
(datetime.datetime(2018, 11, 29, 0, 7, 13, 796000), 0, 29, 48, 11, 2018, 'Thursday')
(datetime.datetime(2018, 11, 29, 0, 10, 56, 796000), 0, 29, 48, 11, 2018, 'Thursday')
(datetime.datetime(2018, 11, 29, 0, 11, 4, 796000), 0, 29, 48, 11, 2018, 'Thursday')
(datetime.datetime(2018, 11, 29, 0, 14, 10, 796000), 0, 29, 48, 11, 2018, 'Thursday')
(datetime.datetime(2018, 11, 29, 0, 16, 2, 796000), 0, 29, 48, 11, 2018, 'Thursday')
(datetime.datetime(2018, 11, 29, 0, 18, 11, 796000), 0, 29, 48, 11, 2018, 'Thursday')
(datetime.datetime(2018, 11, 29, 0, 22, 13, 796000), 0, 29, 48, 11, 2018, 'Thursday')
(datetime.datetime(2018, 11, 29, 0, 26, 15, 796000), 0, 29, 48

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [34]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [35]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df.head()

,userId,firstName,lastName,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid
2,24,Layla,Griffin,F,paid
3,73,Jacob,Klein,M,paid
4,24,Layla,Griffin,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [36]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

In [37]:
# Check if all rows inserted in the relevant table as expected
cur.execute("select * from users;")
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(73, 'Jacob', 'Klein', 'M', 'paid')
(50, 'Ava', 'Robinson', 'F', 'free')
(4, 'Alivia', 'Terrell', 'F', 'free')
(24, 'Layla', 'Griffin', 'F', 'paid')
(54, 'Kaleb', 'Cook', 'M', 'free')
(14, 'Theodore', 'Harris', 'M', 'free')
(98, 'Jordyn', 'Powell', 'F', 'free')
(55, 'Martin', 'Johnson', 'M', 'free')
(78, 'Chloe', 'Roth', 'F', 'free')
(101, 'Jayden', 'Fox', 'M', 'free')
(10, 'Sylvie', 'Cruz', 'F', 'free')
(52, 'Theodore', 'Smith', 'M', 'free')
(12, 'Austin', 'Rosales', 'M', 'free')
(49, 'Chloe', 'Cuevas', 'F', 'paid')
(9, 'Wyatt', 'Scott', 'M', 'free')
(39, 'Walter', 'Frye', 'M', 'free')
(75, 'Joseph', 'Gutierrez', 'M', 'free')
(22, 'Sean', 'Wilson', 'F', 'free')
(32, 'Lily', 'Burns', 'F', 'free')
(79, 'James', 'Martin', 'M', 'free')
(74, 'Braden', 'Parker', 'M', 'free')
(44, 'Aleena', 'Kirby', 'F', 'paid')
(26, 'Ryan', 'Smith', 'M', 'free')
(80, 'Tegan', 'Levine', 'F', 'paid')
(89, 'Kynnedi', 'Sanchez', 'F', 'free')
(82, 'Avery', 'Martinez', 'F', 'paid')
(16, 'Rylan', 'George', 'M', 'p

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [40]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [41]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    # start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
    songplay_data = (pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

In [42]:
# Check if all rows inserted in the relevant table as expected
cur.execute("select * from songplays;")
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, datetime.datetime(2018, 11, 29, 0, 0, 57, 796000), 73, 'paid', None, None, 954, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
(2, datetime.datetime(2018, 11, 29, 0, 1, 30, 796000), 24, 'paid', None, None, 984, 'Lake Havasu City-Kingman, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
(3, datetime.datetime(2018, 11, 29, 0, 4, 1, 796000), 24, 'paid', None, None, 984, 'Lake Havasu City-Kingman, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
(4, datetime.datetime(2018, 11, 29, 0, 4, 55, 796000), 73, 'paid', None, None, 954, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
(5, datetime.datetime(2018, 11, 29, 0, 7, 13, 7

# Close Connection to Sparkify Database

In [31]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.